In [ ]:
# 这个notebook是用来把设计表里平铺的数据，散落/更新/添加/修改到项目的配置表里
# 这张表格是设计表： intermediateTables\PreCATFishesOrigin.xlsx
# 我们要使用它，来向 configs\xlsx_channel\2001\fish_env_affinity.xlsx 以及 configs\xlsx_channel\2001\fish_pond.xlsx 中添加/修改数据
# 数据应该是单向流动的，尽量不要修改configs目录下的表格
import os.path

rootPath = ".."
ORIGIN_TABLE = r"intermediateTables\PreCATFishesOrigin.xlsx"
AffinitySpreadSheet = r"configs\xlsx_channel\2001\fish_env_affinity.xlsx"
PondSpreadSheet = r"configs\xlsx_channel\2001\fish_pond.xlsx"

ORIGIN_TABLE = os.path.join(rootPath, ORIGIN_TABLE)
AffinitySpreadSheet = os.path.join(rootPath, AffinitySpreadSheet)
PondSpreadSheet = os.path.join(rootPath, PondSpreadSheet)

sheetOrigin = "鱼种设计表"
sheetDictQuality = "品质字典"

startColumnNameStruct = "|调整后遮蔽"
startColumnNameBait = "|饵"
startColumnNameWaterLayer = "|觅食层"
startColumnNameTemp = "|温度"

columnNameFishSpeci = "Species"

dictTempColumns = {
    "中间温":"temperature_fav",
    "调整后温度耐受":"temp_affected_ratio",

In [15]:
import pandas as pd

OriginDf = pd.read_excel(ORIGIN_TABLE, sheet_name="鱼种设计表")
# OriginDf.head()
OriginDf.columns

dictQualityDf = pd.read_excel(ORIGIN_TABLE, sheet_name=sheetDictQuality)


In [16]:
# 把columns切成几段，以那些|开头的为分割线
def split_columns(columns):
    split_columns = []
    current_segment = []
    for col in columns:
        # 强制转为字符串再判断前缀，避免非 str 类型（如 int）报错
        if str(col).startswith("|"):
            if current_segment:
                split_columns.append(current_segment)
            current_segment = [col]
        else:
            current_segment.append(col)
    if current_segment:
        split_columns.append(current_segment)
    return split_columns

# print the segments
splittedCols = split_columns(OriginDf.columns)
print(splittedCols)

for segment in splittedCols:
    print(segment)

[['Species', '鱼种', '是否保底', '等阶', '细化等阶', '备注', '本阶内单条价格', '品质', '地图', '阶内权重', 'xp参考', 'gold参考'], ['|温度', '舒适温度min', '舒适温度max', '中间温', '温度耐受', '温度耐受调整', '调整后温度耐受', '温度耐受阈值'], ['|遮蔽', '[水下结构体]开放水域', '[水下结构体]水草', '[水下结构体]石头', '[水下结构体]沉木', '[水下结构体]桥墩'], ['|调整后遮蔽', '[水下结构体]开放水域.1', '[水下结构体]水草.1', '[水下结构体]石头.1', '[水下结构体]沉木.1', '[水下结构体]桥墩.1'], ['|觅食层', '[地图水层]表层', '[地图水层]中层', '[地图水层]底层'], ['|饵', '[真饵]昆虫', '[真饵]甲壳', '[真饵]鱼饵', '[真饵]鱼块', '[真饵]鱼卵', '[真饵]面团', '[真饵]谷物', '[拟饵]T尾', '[拟饵]卷尾', '[拟饵]软虫', '[拟饵]虾管', '[拟饵]米诺', '[拟饵]波爬', '[拟饵]勺子亮片', '[拟饵]旋转亮片', '[拟饵]VIB', '[拟饵]水面拖拉机', '[拟饵]铅笔'], ['|其他', '气压敏感度'], ['|推演水温亲和', '摄氏度下亲和度', 10, 12, 14, 16, 18, 20, 22, 24, 26]]
['Species', '鱼种', '是否保底', '等阶', '细化等阶', '备注', '本阶内单条价格', '品质', '地图', '阶内权重', 'xp参考', 'gold参考']
['|温度', '舒适温度min', '舒适温度max', '中间温', '温度耐受', '温度耐受调整', '调整后温度耐受', '温度耐受阈值']
['|遮蔽', '[水下结构体]开放水域', '[水下结构体]水草', '[水下结构体]石头', '[水下结构体]沉木', '[水下结构体]桥墩']
['|调整后遮蔽', '[水下结构体]开放水域.1', '[水下结构体]水草.1', '[水下结构体]石头.1', '[水下结构体]沉木.1', '[水下结构体]桥墩.1']
['|觅食层'

In [17]:
# 从splittedCols里取出以startColumnNameStruct开头的segment,并去除startColumnNameStruct这一列
# 然后动态的遍历修改AffinitySpreadSheet中的数据
def get_segment_by_start_name(splittedCols, startName):
    for segment in splittedCols:
        if segment[0].startswith(startName):
            return segment[1:]
    return None

# 取出以|遮蔽开头的segment,并去除|遮蔽这一列
segmentStruct = get_segment_by_start_name(splittedCols, startColumnNameStruct)
print(segmentStruct)


['[水下结构体]开放水域.1', '[水下结构体]水草.1', '[水下结构体]石头.1', '[水下结构体]沉木.1', '[水下结构体]桥墩.1']


In [23]:
# 开始遍历修改AffinitySpreadSheet中的数据
import openpyxl

openpyxl.open(AffinitySpreadSheet, read_only=False, keep_links=False)
AffinityPyxl = openpyxl.load_workbook(AffinitySpreadSheet)
# 温度sheet的名字是“TempAffinity”
tempAffinitySheetPath = AffinityPyxl["TempAffinity"]
structAffinitySheetPath = AffinityPyxl["StructAffinity"]
baitTypeAffinitySheetPath = AffinityPyxl["BaitTypeAffinity"]
topAffinitySheetPath = AffinityPyxl["FishEnvAffinity"]


In [24]:
import openpyxl

openpyxl.open(AffinitySpreadSheet, read_only=False, keep_links=False)
AffinityPyxl = openpyxl.load_workbook(AffinitySpreadSheet)

topAffinitySheet = AffinityPyxl["FishEnvAffinity"]

fishNames = OriginDf[columnNameFishSpeci].tolist()

def getFishNameRowFromTopAffinitySheet(fishName) -> int:
    prefixedFishName = f"Fish_{fishName}"
    foundFishName = False
    # 在表格中B列查找这个名字
    for row in range(2, topAffinitySheet.max_row + 1):
        cell = topAffinitySheet.cell(row=row, column=2)
        if cell.value == prefixedFishName:
            foundFishName = True
            return row
            break
    if not foundFishName:
        lastId = topAffinitySheet.cell(row=topAffinitySheet.max_row, column=1).value
        if lastId is None:
            lastId = 1010300
        else:
            newId = lastId + 10 - lastId % 10

        # add a new row, and set the fish name in B column
        new_row = topAffinitySheet.max_row + 1
        topAffinitySheet.cell(row=new_row, column=2, value=prefixedFishName)
        topAffinitySheet.cell(row=new_row, column=1, value=newId)
        # set the fish name in the 3rd column
        topAffinitySheet.cell(row=new_row, column=3, value=fishName)
        return new_row
    return -1
        
for fishName in fishNames:
    getFishNameRowFromTopAffinitySheet(fishName)

## 水下结构体数据格式转换
将源表中的宽格式数据（每行一种鱼有多列不同结构体的数值）转换成目标表中的结构化格式（每行一种鱼，结构体类型和系数交替排列的多列）

In [ ]:
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows

# 使用前面定义的StructuredExcel类处理特殊结构
class StructuredExcel:
    """处理带有特殊行的Excel文件：
       第1行: 数据类型（INT64, STRING等）
       第2行: 英文列名（作为DataFrame的列名）
       第3行: 中文列名
       第4行: 数据结构标记（[List]#1等）
       第5行+: 实际数据行
    """
    
    def __init__(self, excel_path):
        self.excel_path = excel_path
        self.workbook = None
        self._load_workbook()
    
    def _load_workbook(self):
        self.workbook = openpyxl.load_workbook(self.excel_path)
    
    def get_sheet_names(self):
        """获取所有工作表名称"""
        return self.workbook.sheetnames
    
    def read_data(self, sheet_name):
        """读取数据部分，返回DataFrame，跳过类型、中文名和标记行"""
        return pd.read_excel(self.excel_path, sheet_name=sheet_name, header=0, skiprows=[0, 2, 3])
    
    def read_metadata(self, sheet_name):
        """读取元数据行"""
        # 读取类型行（第1行）
        types_row = pd.read_excel(self.excel_path, sheet_name=sheet_name, header=None, nrows=1)
        # 读取中文名行（第3行）
        chinese_row = pd.read_excel(self.excel_path, sheet_name=sheet_name, header=None, skiprows=2, nrows=1)
        # 读取标记行（第4行）
        marker_row = pd.read_excel(self.excel_path, sheet_name=sheet_name, header=None, skiprows=3, nrows=1)
        
        return {
            'types': types_row.iloc[0].to_dict(),
            'chinese_names': chinese_row.iloc[0].to_dict(),
            'markers': marker_row.iloc[0].to_dict()
        }

# 定义宽格式转结构化格式的函数
def convert_wide_to_structured(source_df, struct_columns):
    """
    将源表中的宽格式数据转换为目标表中的结构化格式
    
    参数:
    - source_df: 源DataFrame，包含鱼类和结构体系数
    - struct_columns: 水下结构体列名列表
    
    返回:
    - 转换后的DataFrame，适合写入目标表格
    """
    # 获取鱼种名称列 (如果存在)
    fish_name_col = 'name' if 'name' in source_df.columns else 'Species'
    
    # 准备结果DataFrame
    result_rows = []
    
    # 为每个鱼种创建一行
    for idx, row in source_df.iterrows():
        # 开始构建新行
        new_row = {}
        
        # 添加ID和鱼名 (如果存在)
        if 'id' in source_df.columns:
            new_row['id'] = row['id']
        if fish_name_col in source_df.columns:
            new_row['name'] = row[fish_name_col]
        
        # 为每个结构体类型添加一对struct_type和coeff列
        for i, struct_type in enumerate(struct_columns):
            # 确定struct_type列和coeff列的名称
            type_col = f'struct_type{"" if i == 0 else f".{i}"}'  
            coeff_col = f'coeff{"" if i == 0 else f".{i}"}'
            
            # 将结构类型名称和系数值放入对应列
            new_row[type_col] = struct_type
            new_row[coeff_col] = float(row[struct_type])
        
        # 添加备注列 (如果存在)
        if 'mark' in source_df.columns:
            new_row['mark'] = row['mark']
        elif '备注' in source_df.columns:
            new_row['mark'] = row['备注']
        
        # 将新行添加到结果中
        result_rows.append(new_row)
    
    # 创建结果DataFrame
    result_df = pd.DataFrame(result_rows)
    return result_df

# # 示例使用
# # 1. 定义源数据
# example_data = {
#     'Species': ['Tench', 'Golden_Bream'],
#     '[水下结构体]开放水域': [0.0, 0.0],
#     '[水下结构体]水草': [0.6, 1.0],
#     '[水下结构体]石头': [0.6, 0.0],
#     '[水下结构体]沉木': [1.0, 0.0],
#     '[水下结构体]桥墩': [0.0, 0.0]
# }
# example_df = pd.DataFrame(example_data)

# # 2. 定义结构体列名
# struct_columns = ['[水下结构体]开放水域', '[水下结构体]水草', '[水下结构体]石头', '[水下结构体]沉木', '[水下结构体]桥墩']

# # 3. 转换数据
# converted_df = convert_wide_to_structured(example_df, struct_columns)

# # 4. 显示结果
# print("源数据:")
# print(example_df)
# print("\n转换后的结构化数据:")
# print(converted_df)

In [21]:
# 将OriginDf中的水下结构体数据转换为目标格式并保存

# 1. 从设计表中提取水下结构体列
struct_columns = segmentStruct  # 使用之前提取的结构体列名
print("水下结构体列名:", struct_columns)

# 2. 转换为结构化格式
structured_df = convert_wide_to_structured(OriginDf, struct_columns)

# 3. 检查结果
print("\n转换后的前5行:")
print(structured_df.head())

# 4. 保存到Excel文件（可选）
output_path = os.path.join(rootPath, "intermediateTables", "StructuredEnvData.xlsx")
structured_df.to_excel(output_path, sheet_name='StructAffinity', index=False)
print(f"\n已保存结构化数据到: {output_path}")

# 5. 更新到目标环境亲和度表格（需谨慎操作）
update_target = False  # 设置为True以更新目标表
if update_target:
    # 加载目标Excel
    target_excel = StructuredExcel(AffinitySpreadSheet)
    
    # 读取目标表的元数据
    metadata = target_excel.read_metadata('StructAffinity')
    
    print("\n目标表的列名：", )

    # 打印目标表的元数据以供参考
    print("\n目标表的元数据:")
    print("数据类型:", metadata['types'])
    print("中文列名:", metadata['chinese_names'])
    print("列标记:", metadata['markers'])
    
    # 将转换后的数据添加/更新到目标表
    # 打开目标工作簿
    workbook = openpyxl.load_workbook(AffinitySpreadSheet)
    sheet = workbook['StructAffinity']
    
    # 定义列结构（确保顺序与目标表一致）
    # structured_df = structured_df[[col for col in sheet.cell(row=2, column=1).value]]
    
    # 将数据添加到第5行开始（保留元数据行）
    for r_idx, originRow in enumerate(dataframe_to_rows(structured_df, index=False, header=False)):
        for c_idx, value in enumerate(originRow):
            sheet.cell(row=r_idx+5, column=c_idx+1, value=value)
    
    # 保存更新后的工作簿
    workbook.save(AffinitySpreadSheet)
    print(f"\n已更新数据到目标表: {AffinitySpreadSheet}")
else:
    print("\n未更新到目标表。如需更新，请将update_target设置为True")

水下结构体列名: ['[水下结构体]开放水域.1', '[水下结构体]水草.1', '[水下结构体]石头.1', '[水下结构体]沉木.1', '[水下结构体]桥墩.1']

转换后的前5行:
            name    struct_type  coeff struct_type.1  coeff.1 struct_type.2  \
0          Tench  [水下结构体]开放水域.1    0.0   [水下结构体]水草.1      0.0   [水下结构体]石头.1   
1   Golden_Bream  [水下结构体]开放水域.1    0.0   [水下结构体]水草.1      0.0   [水下结构体]石头.1   
2  Green_Sunfish  [水下结构体]开放水域.1    0.6   [水下结构体]水草.1      1.0   [水下结构体]石头.1   
3  Black_Crappie  [水下结构体]开放水域.1    0.6   [水下结构体]水草.1      1.0   [水下结构体]石头.1   
4  White_Crappie  [水下结构体]开放水域.1    0.6   [水下结构体]水草.1      1.0   [水下结构体]石头.1   

   coeff.2 struct_type.3  coeff.3 struct_type.4  coeff.4  \
0      0.0   [水下结构体]沉木.1      1.0   [水下结构体]桥墩.1      0.0   
1      0.0   [水下结构体]沉木.1      1.0   [水下结构体]桥墩.1      0.0   
2      0.6   [水下结构体]沉木.1      0.0   [水下结构体]桥墩.1      0.0   
3      0.6   [水下结构体]沉木.1      0.0   [水下结构体]桥墩.1      0.0   
4      0.6   [水下结构体]沉木.1      0.6   [水下结构体]桥墩.1      0.0   

                      mark  
0                      NaN  
1  在地图中间弄

In [22]:
workbook = openpyxl.load_workbook(AffinitySpreadSheet)
sheet = workbook['StructAffinity']

for r_idx, originRow in enumerate(dataframe_to_rows(structured_df, index=False, header=False)):
    print(f"Row : ", originRow)
    targetRowName = "Cover_" + originRow[0]
    print(targetRowName)
    headlessRow = originRow[1:-1] # 去掉行名和最后一列的备注
    print(headlessRow)
    # 在第二列查找行名
    for cell in sheet["B"]:
        if cell.value == targetRowName:
            # 找到行名，更新数据,更新的方法是将cell对应的行里，0、1元素不变，2（即第三个元素）号元素起，跟headlessRoww一样长的元素，使用headlessRow的值
            n = len(headlessRow)
            for i in range(n):
                sheet.cell(row=cell.row, column=i+2, value=headlessRow[i])
            break
    else:
        # 如果没有找到行名，则添加新行, and keep the added row as a reference
        # 这里的row[1:]是去掉了行名的部分
        # 这一行的第一个数据是id，用上一行的id往上涨，涨到一个最近的能被10整除的数；
        # 也就是说，id是10的倍数
        maxRow = sheet.max_row
        lastId = sheet.cell(maxRow, 1).value
        if lastId is None:
            lastId = 2010000
        else:
            lastId = int(lastId)
        newId = lastId + 10 - (lastId % 10)
        # print(lastId)
        # add a new row, and keep the added row as a reference, the first cell is newId, the second cell is the row name, appended are headlessRow
        newRow = [newId, targetRowName] + list(headlessRow)
        sheet.append(newRow)

workbook.save(AffinitySpreadSheet)
workbook.close()

Row :  ['Tench', '[水下结构体]开放水域.1', 0.0, '[水下结构体]水草.1', 0.0, '[水下结构体]石头.1', 0.0, '[水下结构体]沉木.1', 1.0, '[水下结构体]桥墩.1', 0.0, nan]
Cover_Tench
['[水下结构体]开放水域.1', 0.0, '[水下结构体]水草.1', 0.0, '[水下结构体]石头.1', 0.0, '[水下结构体]沉木.1', 1.0, '[水下结构体]桥墩.1', 0.0]
Row :  ['Golden_Bream', '[水下结构体]开放水域.1', 0.0, '[水下结构体]水草.1', 0.0, '[水下结构体]石头.1', 0.0, '[水下结构体]沉木.1', 1.0, '[水下结构体]桥墩.1', 0.0, '在地图中间弄个特殊的地方放，并在水面上做点提示']
Cover_Golden_Bream
['[水下结构体]开放水域.1', 0.0, '[水下结构体]水草.1', 0.0, '[水下结构体]石头.1', 0.0, '[水下结构体]沉木.1', 1.0, '[水下结构体]桥墩.1', 0.0]
Row :  ['Green_Sunfish', '[水下结构体]开放水域.1', 0.6, '[水下结构体]水草.1', 1.0, '[水下结构体]石头.1', 0.6, '[水下结构体]沉木.1', 0.0, '[水下结构体]桥墩.1', 0.0, nan]
Cover_Green_Sunfish
['[水下结构体]开放水域.1', 0.6, '[水下结构体]水草.1', 1.0, '[水下结构体]石头.1', 0.6, '[水下结构体]沉木.1', 0.0, '[水下结构体]桥墩.1', 0.0]
Row :  ['Black_Crappie', '[水下结构体]开放水域.1', 0.6, '[水下结构体]水草.1', 1.0, '[水下结构体]石头.1', 0.6, '[水下结构体]沉木.1', 0.0, '[水下结构体]桥墩.1', 0.0, nan]
Cover_Black_Crappie
['[水下结构体]开放水域.1', 0.6, '[水下结构体]水草.1', 1.0, '[水下结构体]石头.1', 0.6, '[水下结构体]沉木.1', 0

PermissionError: [Errno 13] Permission denied: '..\\configs\\xlsx_channel\\2001\\fish_env_affinity.xlsx'